## Modeling Pipeline :
* 1.use item similarity to overcome cold start issue
* 2.pipeline: train model, save model, import new user data, load model for recommendation, output recommendation, 

In [1]:
import boto3
import pickle
import h5py
import tables
import numpy as np
import pandas as pd
import sklearn
import caserec
import json

In [2]:
import turicreate as tc

In [3]:
#raw_data=pd.read_csv('training_data_3column_750.csv')

In [4]:
#raw_data_user_count=raw_data.groupby('game_id')['user_id'].count().reset_index()

In [5]:
#training_data_2k=raw_data[raw_data.game_id.isin(raw_data_user_count[raw_data_user_count.user_id>2000].game_id)]

In [6]:
#training_data_2k.shape

In [7]:
#training_data_2k.game_id.nunique()

In [8]:
#training_data_2k.to_csv('training_data_3column_2k_new.csv')

In [15]:
#actions = tc.SFrame.read_csv('training_data_3column_2k_new.csv')
actions = tc.SFrame.read_csv('training_data_3column_750.csv')
items = tc.SFrame.read_csv('game_info_750.csv')
df_items=pd.read_csv('game_info_750.csv')

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/training_data_3column_750.csv

Parsing completed. Parsed 100 lines in 1.89605 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2112641 lines. Lines per second: 1.11942e+06

Read 13521118 lines. Lines per second: 1.81779e+06

...

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 100 lines in 0.05078 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,list,float,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 9595 lines in 0.042817 secs.

In [10]:
actions=actions[['game_id','user_id','rating']]
actions

game_id,user_id,rating
1,Lil Red Head,7.0
1,hsjx945,8.0
1,RuffRyder,0.0
1,SkywalterDBZ,7.0
1,Olmestig,8.0
1,grassquit,9.0
1,erzengel,0.0
1,Feeble,0.0
1,thisisadamj,0.0
1,Tiberiusfox,9.0


In [11]:


model = tc.factorization_recommender.create(actions,'user_id','game_id',target='rating')



Preparing data set.

Data has 40096849 observations with 486469 users and 9595 items.

Data prepared in: 32.778s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 5.09448

Final training RMSE: 2.25332

In [12]:
#save model

model.save('game_rec_model_750_bgguser')

In [3]:
#load model
rec_model=tc.load_model('game_rec_model_750_bgguser')

In [4]:
rec_model

Class                            : FactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : game_id
Target                           : rating
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 40096849
Number of users                  : 486469
Number of items                  : 9595

Training summary
----------------
Training time                    : 203.1504

Model Parameters
----------------
Model class                      : FactorizationRecommender
num_factors                      : 8
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 50

Regularization Settings
-----------------------
regularization                   : 0.0
regularization_type              : normal
linea

In [5]:
#try with random user input data
#new_obs_data = tc.SFrame({ 'user_id' : ['survey', 'survey', 'survey', 'survey', 'survey'],
                               # 'game_id' : ['4098', '9216', '475', '148228', '68448']})

new_obs_data = ['heronymus']


#recommend_games = rec_model.recommend_from_interactions([4098,9216,475])
recommend_games = rec_model.recommend(new_obs_data,k=50)
recommend_games

user_id,game_id,score,rank
heronymus,55690,2.0677386552841184,1
heronymus,269385,1.8388010293991086,2
heronymus,239188,1.77513039417572,3
heronymus,227456,1.7352083951980588,4
heronymus,233078,1.7265000373870847,5
heronymus,210232,1.6610238582641599,6
heronymus,233398,1.5882482559234616,7
heronymus,222514,1.5218483240158078,8
heronymus,170771,1.4969415695220945,9
heronymus,193512,1.4417468339950559,10


In [6]:
new_obs_data1 =  ['Mikorax']
recommend_games = rec_model.recommend(new_obs_data1,k=50)
recommend_games

user_id,game_id,score,rank
Mikorax,227456,1.1949810416252133,1
Mikorax,178944,1.154395940974426,2
Mikorax,55690,1.1376448900253293,3
Mikorax,195162,1.0309503109008786,4
Mikorax,194923,0.9954189092666623,5
Mikorax,170771,0.9551491767913816,6
Mikorax,239188,0.9356907636672971,7
Mikorax,269385,0.8906069428474424,8
Mikorax,181176,0.8850540549308774,9
Mikorax,147887,0.8098241717369077,10


In [341]:
##backend flask import new user data and return 
from flask import Flask
from flask import request
app = Flask(__name__)

import turicreate as tc

import json

rec_model=tc.load_model('game_rec_model')

@app.route("/recommend", methods=['POST'])

SyntaxError: unexpected EOF while parsing (<ipython-input-341-726a4f329022>, line 12)

In [26]:
#function to import json of user's answers, run model prediction, 
def recommend():
#import new user survey answer
#new_obs_data = json.loads("../data/bgg_survey_result.json")

    json_file_path = "../data/bgg_survey_result.json"

    with open(json_file_path, 'r') as j:
         new_obs_data = json.loads(j.read())

    new_obs_data_new=[new_obs_data["user_id"]]
    print(new_obs_data_new)
    filter_condt=tc.SFrame({"age_min": [new_obs_data["age"]["min"]],
                        "age_max": [new_obs_data["age"]["max"]],
                        "num_players_min": [new_obs_data["num_players"]["min"]],
                        "num_players_max": [new_obs_data["num_players"]["max"]],
                        "play_time_min": [new_obs_data["play_time"]["min"]],
                        "play_time_max": [new_obs_data["play_time"]["max"]]})
    recommend_items = rec_model.recommend(new_obs_data_new,k=50)
    
    #select 50 recommended game info 
    df_rec_game_info=df_items.loc[df_items['game_id'].isin(recommend_items['game_id'])]
    #filter out game based on user answers 
    df_items_filter=df_rec_game_info[(df_rec_game_info['age_min'] > filter_condt['age_min'])&
                                #(df_rec_game_info['min_ppl'] < filter_condt['num_players_min']) &
                                #(df_rec_game_info['max_ppl'] < filter_condt['num_players_max']) ]
                                (df_rec_game_info['avg_time']> filter_condt['play_time_min'])&(df_rec_game_info['avg_time']< filter_condt['play_time_max'])]
    #conver recommend_items to dataframe

    df_recommend_items=recommend_items.to_dataframe()
    #output top 5 games
    output=df_recommend_items.loc[df_recommend_items['game_id'].isin(df_items_filter.game_id)].sort_values('score',ascending=False).head(5)
    json_output=json.dumps({"game_id": list(output["game_id"]), "score": list(output["score"])})
    return json_output

In [27]:
recommend()

['Stupendous12']


'{"game_id": [30549, 822, 9209, 14996, 28143], "score": [4.84644878216095, 4.5989512235672, 4.38850725002594, 4.229442480281067, 4.126034143641663]}'